## Projeto 2 - Ciência dos Dados

**Feito por:**

Cesar Ades

Diogo Duarte

Jonathan Sutton

Joras Oliveira

## Introdução

Para o Projeto 2 da disciplina de Ciência dos Dados, decidimos trabalhar com um target qualitativo, que classificaria se a música seria um “hit” ou não, com o critério de ela ter aparecido na lista semanal top 100 da empresa Billboard entre os anos de 2010 até 2019. Para chegar ao resultado final, utilizaremos um classificador com uma base de dados de aproximadamente 6400 músicas.

In [13]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import math

In [10]:
import chardet
with open('dataset-of-10s.csv', 'rb') as rawdata: # Abre o dataset
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [26]:
dados = pd.read_csv('dataset-of-10s.csv',encoding='utf-8') # Faz a leitura do Dataset.

del dados['uri'] # Coluna de identificação de músicas do Spotify removida, pois são valores únicos que não influenciam no trabalho.

dados.head(50) # Imprime as primeiros 50 músicas do Dataset.

,track,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Wild Things,Alessia Cara,0.741,0.6260,1,-4.826,0,0.0886,0.020000,0.000000,0.0828,0.7060,108.029,188493,4,41.18681,10,1
1,Surfboard,Esquivel!,0.447,0.2470,5,-14.661,0,0.0346,0.871000,0.814000,0.0946,0.2500,155.489,176880,3,33.18083,9,0
2,Love Someone,Lukas Graham,0.550,0.4150,9,-6.557,0,0.0520,0.161000,0.000000,0.1080,0.2740,172.065,205463,4,44.89147,9,1
3,Music To My Ears (feat. Tory Lanez),Keys N Krates,0.502,0.6480,0,-5.698,0,0.0527,0.005130,0.000000,0.2040,0.2910,91.837,193043,4,29.52521,7,0
4,Juju On That Beat (TZ Anthem),Zay Hilfigerrr & Zayion McCall,0.807,0.8870,1,-3.892,1,0.2750,0.003810,0.000000,0.3910,0.7800,160.517,144244,4,24.99199,8,1
5,Here's To Never Growing Up,Avril Lavigne,0.482,0.8730,0,-3.145,1,0.0853,0.011100,0.000000,0.4090,0.7370,165.084,214320,4,32.17301,12,1
6,Sex Metal Barbie,In This Moment,0.533,0.9350,0,-3.704,1,0.1280,0.013900,0.000000,0.1680,0.4810,140.092,262493,4,21.04510,14,0
7,Helluva Night,Ludacris,0.736,0.5220,2,-8.020,1,0.1160,0.029900,0.000000,0.1080,0.3690,97.547,200387,4,60.21027,10,1
8,Holiday With HH,No Bros,0.166,0.9850,7,-2.886,1,0.1700,0.001830,0.014200,0.9580,0.1390,174.725,252787,4,31.23583,11,0
9,My Last,Big Sean Featuring Chris Brown,0.387,0.7730,8,-5.685,1,0.1700,0.098000,0.000000,0.2090,0.3680,78.629,254120,4,23.30245,9,1


In [27]:
#Quantidade de Musicas
index = dados. index
number_of_rows = len(index)
print(f'Este Dataset tem:  {number_of_rows}' )

Este Dataset tem:  6398
